In [1]:
import os, sys, inspect
sys.path.insert(1, os.path.join(sys.path[0], './conformal_classification/'))

import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as tdata
from conformal import *
from utils import *

def read_file(File='datasets2022.npz'):
    dataset = np.load(File)
    # 转换数据形状为 [N, 1, 52, 52]
    train_data = dataset['train'].transpose(0, 3, 1, 2)  # 从 [N,52,52,1] 转为 [N,1,52,52]
    label_train = dataset['label_train'].astype(np.float32)
    return torch.Tensor(train_data), torch.Tensor(label_train)

# 加载数据
train_data, label_train = read_file()

# 只取前1000个样本进行测试
train_data = train_data[:1000]
label_train = label_train[:1000]

# 创建数据集
full_dataset = tdata.TensorDataset(train_data, label_train)

# 划分校准集和验证集
n_data_conf = int(len(full_dataset) * 0.7)  # 70% 作为校准集
n_data_val = len(full_dataset) - n_data_conf
logits_cal, logits_val = split2(full_dataset, n_data_conf, n_data_val)

# 创建数据加载器
bsz = 32
loader_cal = tdata.DataLoader(logits_cal, batch_size = bsz, shuffle=False, pin_memory=True)
loader_val = tdata.DataLoader(logits_val, batch_size = bsz, shuffle=False, pin_memory=True)

class ExampleModel(nn.Module):
    def __init__(self):
        super().__init__()
        # 输入形状应为 [batch, 1, 52, 52]
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, 3),      # [batch, 16, 50, 50]
            nn.ReLU(),
            nn.MaxPool2d(2),          # [batch, 16, 25, 25]
            nn.Conv2d(16, 32, 3),     # [batch, 32, 23, 23]
            nn.ReLU(),
            nn.MaxPool2d(2)           # [batch, 32, 11, 11]
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),             # [batch, 32 * 11 * 11 = 3872]
            nn.Linear(3872, 512),
            nn.ReLU(),
            nn.Linear(512, 8)         # 输出8个logits对应8个标签
        )

    def forward(self, x):
        x = self.conv(x)
        return self.classifier(x)  # 输出形状 [batch, 8]

model = ExampleModel()
model = model.cuda()  # 若有GPU可用

# 初始化共形模型
alpha = 0.1  # 期望的覆盖率
kreg = None
lamda = None
randomized = True
allow_zero_sets = True
pct_paramtune = 0.3
lamda_criterion = 'size'

conformal_model = ConformalModelMulti(model, loader_cal, alpha=alpha, kreg=kreg, lamda=lamda, randomized=randomized, allow_zero_sets=allow_zero_sets, pct_paramtune=pct_paramtune, lamda_criterion=lamda_criterion)

# 验证模型
cvg_avg, sz_avg = validate(loader_val, conformal_model, print_bool=True)

print(f"Coverage: {cvg_avg:.3f}")
print(f"Average Set Size: {sz_avg:.3f}")

Coverage: 0.563
Average Set Size: 6.827
